In [1]:
import numpy as np
import pandas as pd

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
def infer(model, test_dir, train_dir, batch_size):
    test_datagenerator = ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        class_mode=None,
        target_size=(299, 299),
        batch_size=batch_size,
        shuffle=False,
    )
    
    filenames = test_generator.filenames
    samples = len(filenames)

    labels = ImageDataGenerator().flow_from_directory(train_dir).class_indices
    labels = dict((v, k) for k, v in labels.items())
    #print(labels)
    
    predictions = model.predict_generator(test_generator, steps=samples//batch_size)
    predicted_class_indices = np.argmax(predictions, axis=1)

    predictions = [labels[k] for k in predicted_class_indices]

    return (predictions, [filename.replace('no_labels/', '') for filename in filenames])

In [3]:
# == VARIABLES ==
train_dir = './data/kfold/0/train'
test_dir = './data/test'
batch_size = 32

In [4]:
# == PREDCIT ==
# Get model's architecture from json file
with open('./models/picnic_model_v3.json', 'r') as file:
    json_string = file.read()

# Load model's architecture
loaded_model = model_from_json(json_string)
# Load model's weight
loaded_model.load_weights('./models/picnic_model_v3.h5')

In [5]:
(predictions, filenames) = infer(loaded_model, test_dir, train_dir, batch_size)
results = pd.DataFrame({"file": filenames, "label": predictions})
results.to_csv("14-04-19_1.tsv", sep='\t', index=False)